# Build a qualitative researcher & launch an interview
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for simulating qualitative research interviews. In a series of steps we construct an AI agent representing a qualitative researcher and launch an interview with specified topics and interview subjects.

EDSL is an open-source library for simulating surveys and experiments with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/qualitative_research.ipynb)

Importing tools:

In [1]:
from edsl.questions import QuestionFreeText
from edsl import Agent, Scenario, Model

import textwrap
from rich import print

EDSL works with many popular language models ([learn more](https://docs.expectedparrot.com/en/latest/language_models.html) about using models). Here we select one to use in running in our survey:

In [2]:
model = Model("gpt-4-1106-preview")

Here we create methods for constructing agents representing a researcher and subject, and conducting an interview in the form of a survey that we can observe. (Learn more about [designing agents](https://docs.expectedparrot.com/en/latest/agents.html) and [running surveys](https://docs.expectedparrot.com/en/latest/surveys.html)).

In [3]:
def construct_subject(name, traits = {}):
    return Agent(name = name, traits = traits)
 
def construct_researcher(product, topics):
    background = textwrap.dedent(f"""\
    You are an expert qualitative researcher. You are researching this product: {product}. 
    You are interested in conducting interviews to learn people's views on the following topics relating 
    to the product: {topics}.""")

    instruction = textwrap.dedent(f"""\
    You know to ask questions that are appropriate to the age and experience of an interview subject.
    You know to not ask questions that an interview subject would not be able to answer, 
    e.g., if they are a young child, they probably will not be able to answer many questions about prices. 
    You ask excellent follow-up questions.    
    """)
    return Agent(traits = {'background': background}, instruction = instruction)

def get_next_question(subject, researcher, dialog_so_far):
    scenario = Scenario({'subject': str(subject.traits), 'dialog_so_far': dialog_so_far})
    meta_q = QuestionFreeText(
        question_name = "next_question",
        question_text = """
        These are the biographic details of the interview subject: {{ subject }}
        This is your current dialog with the interview subject: {{ dialog_so_far }}
        What question you would ask the interview subject next?
        """
    )
    question_text = meta_q.by(model).by(researcher).by(scenario).run().select("next_question").first()
    return question_text

def get_response_to_question(question_text, subject, dialog_so_far):
    q_to_subject = QuestionFreeText(
        question_name = "question",
        question_text = f"""
        This is your current dialog with the interview subject: {dialog_so_far}. 
        You are now being asked:""" + question_text 
    )
    response = q_to_subject.by(model).by(subject).run().select("question").first() 
    return response

def ask_question(subject, researcher, dialog_so_far):
    question_text = get_next_question(subject, researcher, dialog_so_far)
    response = get_response_to_question(question_text, subject, dialog_so_far)
    return {"question": question_text, "response": response}

def dialog_to_string(d):
    return "\n".join([f"Question: {d['question']}\nResponse: {d['response']}" for d in d])

In [4]:
subject = construct_subject(name = "Alice", traits = {'gender': "female", 'age': 12})
researcher = construct_researcher(product = "cassette tapes", topics = "technology, music")
result = ask_question(subject, researcher, "")
print(result)

{
    'question': 'What kind of music do you like to listen to, and how do you usually listen to it?',
    'response': "I really like listening to pop music and some K-pop because the songs are super catchy and fun to 
dance to! I usually listen to music on my phone with my headphones or sometimes I play it out loud in my room when 
I'm doing homework or just chilling."
}

In [5]:
dialog_so_far = []
for i in range(5):
    result = ask_question(subject, researcher, dialog_to_string(dialog_so_far))
    print(result)
    dialog_so_far.append(result)
print(dialog_so_far)

{
    'question': 'What kind of music do you like to listen to, and how do you usually listen to it?',
    'response': "I really like listening to pop music and some K-pop because the songs are super catchy and fun to 
dance to! I usually listen to music on my phone with my headphones or sometimes I play it out loud in my room when 
I'm doing homework or just chilling."
}

{
    'question': 'That sounds like a fun way to enjoy your favorite tunes! Do you ever use any other devices besides
your phone to listen to music, like a CD player, radio, or maybe even a cassette player?',
    'response': "Oh, I mostly use my phone because it's super convenient, but I do have this small Bluetooth 
speaker that I connect to sometimes when I want the music to be really loud! My parents have a CD player in the 
living room that we use sometimes when they want to play their old CDs. I've never used a cassette player though – 
that's like, retro, right?"
}

{
    'question': "It's interesting to hear that you enjoy the convenience of your phone for music! Since you 
mentioned that you've never used a cassette player, what do you think about the idea of using older technology like
cassette tapes to listen to music? Do you think it would be fun to try it out, or do you prefer sticking to more 
modern ways of listening to music?",
    'response': "I think it would be kind of cool to try out a cassette player just to see what it was like for 
people to listen to music back then. It's like a little piece of history! I'm really into modern tech because it's 
easy to use and I can listen to whatever I want, whenever I want. But trying a cassette player could be a fun 
experience, even if it's just once or for a special occasion. It's always neat to learn how things used to be!"
}

{
    'question': "That's a great perspective on experiencing history through music technology! If you had the chance
to use a cassette player, what do you think would be the most interesting part about it? Would it be the physical 
aspect of handling tapes, the sound quality, or maybe the nostalgia it represents for earlier times?",
    'response': "I think the most interesting part about using a cassette player would be the physical aspect of 
handling the tapes. It's so different from just tapping on a screen. You get to actually hold the cassette, flip it
over, and push it into the player, which is kind of cool and hands-on. Plus, watching the tape spin inside the 
player must be pretty neat. I guess the sound quality would be interesting to compare too, since I'm used to 
digital music. The nostalgia part is more for my parents, but it would be fun to see what they get excited about!"
}

{
    'question': "That's a thoughtful observation about the tactile experience! If you were to create a mixtape 
using a cassette, what songs would you choose to include on it, and why those particular songs?",
    'response': "If I were to create a mixtape, I'd pick a bunch of my favorite pop and K-pop songs! I would 
include 'Dynamite' by BTS because it's so upbeat and makes me want to dance. Then I'd add 'drivers license' by 
Olivia Rodrigo because I love singing along to it. 'Shake It Off' by Taylor Swift would be on there too because 
it's super catchy and fun. I'd also choose some songs from BLACKPINK, like 'How You Like That,' because their music
is really cool. And of course, I'd have to add some classic pop hits like 'Call Me Maybe' by Carly Rae Jepsen 
because it's a classic and everyone loves it. These songs are all special to me because they're the ones I always 
have on repeat and they just make me really happy whenever I hear them!"
}

[
    {
        'question': 'What kind of music do you like to listen to, and how do you usually listen to it?',
        'response': "I really like listening to pop music and some K-pop because the songs are super catchy and fun
to dance to! I usually listen to music on my phone with my headphones or sometimes I play it out loud in my room 
when I'm doing homework or just chilling."
    },
    {
        'question': 'That sounds like a fun way to enjoy your favorite tunes! Do you ever use any other devices 
besides your phone to listen to music, like a CD player, radio, or maybe even a cassette player?',
        'response': "Oh, I mostly use my phone because it's super convenient, but I do have this small Bluetooth 
speaker that I connect to sometimes when I want the music to be really loud! My parents have a CD player in the 
living room that we use sometimes when they want to play their old CDs. I've never used a cassette player though – 
that's like, retro, right?"
    },
    {
        'question': "It's interesting to hear that you enjoy the convenience of your phone for music! Since you 
mentioned that you've never used a cassette player, what do you think about the idea of using older technology like
cassette tapes to listen to music? Do you think it would be fun to try it out, or do you prefer sticking to more 
modern ways of listening to music?",
        'response': "I think it would be kind of cool to try out a cassette player just to see what it was like for
people to listen to music back then. It's like a little piece of history! I'm really into modern tech because it's 
easy to use and I can listen to whatever I want, whenever I want. But trying a cassette player could be a fun 
experience, even if it's just once or for a special occasion. It's always neat to learn how things used to be!"
    },
    {
        'question': "That's a great perspective on experiencing history through music technology! If you had the 
chance to use a cassette player, what do you think would be the most interesting part about it? Would it be the 
physical aspect of handling tapes, the sound quality, or maybe the nostalgia it represents for earlier times?",
        'response': "I think the most interesting part about using a cassette player would be the physical aspect 
of handling the tapes. It's so different from just tapping on a screen. You get to actually hold the cassette, flip
it over, and push it into the player, which is kind of cool and hands-on. Plus, watching the tape spin inside the 
player must be pretty neat. I guess the sound quality would be interesting to compare too, since I'm used to 
digital music. The nostalgia part is more for my parents, but it would be fun to see what they get excited about!"
    },
    {
        'question': "That's a thoughtful observation about the tactile experience! If you were to create a mixtape 
using a cassette, what songs would you choose to include on it, and why those particular songs?",
        'response': "If I were to create a mixtape, I'd pick a bunch of my favorite pop and K-pop songs! I would 
include 'Dynamite' by BTS because it's so upbeat and makes me want to dance. Then I'd add 'drivers license' by 
Olivia Rodrigo because I love singing along to it. 'Shake It Off' by Taylor Swift would be on there too because 
it's super catchy and fun. I'd also choose some songs from BLACKPINK, like 'How You Like That,' because their music
is really cool. And of course, I'd have to add some classic pop hits like 'Call Me Maybe' by Carly Rae Jepsen 
because it's a classic and everyone loves it. These songs are all special to me because they're the ones I always 
have on repeat and they just make me really happy whenever I hear them!"
    }
]